In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import tensorflow as tf
from pprint import pprint
import cv2

2022-12-04 00:41:38.867352: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 00:41:38.867479: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
ROOT_DIR = os.path.abspath('.')
MODELS_DIR = os.path.join(ROOT_DIR, 'models')
CUSTOM_MODEL = os.path.join(MODELS_DIR, 'experiments', 'integer_only.tflite')
IMAGE_DIR =  os.path.join(ROOT_DIR, 'test')


In [3]:
interpreter = tf.lite.Interpreter(CUSTOM_MODEL)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()
pprint(input_details)
pprint(output_details)

{'dtype': <class 'numpy.int8'>,
 'index': 0,
 'name': 'serving_default_input:0',
 'quantization': (0.003921568859368563, -128),
 'quantization_parameters': {'quantized_dimension': 0,
                             'scales': array([0.00392157], dtype=float32),
                             'zero_points': array([-128], dtype=int32)},
 'shape': array([  1, 320, 320,   3], dtype=int32),
 'shape_signature': array([  1, 320, 320,   3], dtype=int32),
 'sparsity_parameters': {}}
[{'dtype': <class 'numpy.int8'>,
  'index': 386,
  'name': 'StatefulPartitionedCall:1',
  'quantization': (0.00385795789770782, -128),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([0.00385796], dtype=float32),
                              'zero_points': array([-128], dtype=int32)},
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([ 1, 10], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': <class 'numpy.int8'>,
  'index': 384,
  'name': 'Stat

In [5]:
interpreter.get_tensor_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 320, 320,   3], dtype=int32),
  'shape_signature': array([  1, 320, 320,   3], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.003921568859368563, -128),
  'quantization_parameters': {'scales': array([0.00392157], dtype=float32),
   'zero_points': array([-128], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'ssd_mobile_net_v2_fpn_keras_feature_extractor/FeatureMaps/top_down/nearest_neighbor_upsampling/nearest_neighbor_upsampling/Reshape/shape',
  'index': 1,
  'shape': array([4], dtype=int32),
  'shape_signature': array([4], dtype=int32),
  'dtype': numpy.int32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'ssd_mobile_net_v2_fpn_keras_feature_extractor/FeatureMaps/top_down/nearest_neighbor_upsampling/nearest_

In [7]:
img = cv2.imread(os.path.join(IMAGE_DIR, 'Cars401.png'))
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (input_width, input_height))

test_image = np.array(img)
test_image

array([[[203, 211, 220],
        [200, 208, 218],
        [200, 208, 218],
        ...,
        [151, 166, 179],
        [150, 165, 178],
        [155, 169, 182]],

       [[200, 210, 221],
        [198, 207, 218],
        [196, 206, 217],
        ...,
        [149, 164, 177],
        [149, 164, 177],
        [153, 168, 181]],

       [[197, 208, 220],
        [195, 205, 217],
        [194, 205, 217],
        ...,
        [148, 162, 175],
        [148, 162, 175],
        [151, 166, 179]],

       ...,

       [[ 70,  44,  31],
        [ 81,  57,  45],
        [103,  81,  69],
        ...,
        [ 90,  69,  53],
        [ 86,  66,  49],
        [ 84,  64,  47]],

       [[ 81,  60,  45],
        [ 88,  66,  52],
        [ 75,  52,  40],
        ...,
        [ 92,  71,  54],
        [ 92,  70,  54],
        [ 91,  70,  53]],

       [[ 81,  65,  48],
        [ 80,  61,  46],
        [ 63,  41,  28],
        ...,
        [100,  79,  61],
        [103,  82,  64],
        [ 89,  68,  50]]

In [10]:
input_scale, input_zero_point = input_details["quantization"]
test_image = (test_image / input_scale) + input_zero_point
test_image

array([[[51636.99693867, 53676.99681803, 55971.9966823 ],
        [50871.99698391, 52911.99686327, 55461.99671246],
        [50871.99698391, 52911.99686327, 55461.99671246],
        ...,
        [38376.99772285, 42201.99749665, 45516.9973006 ],
        [38121.99773793, 41946.99751173, 45261.99731568],
        [39396.99766253, 42966.99745141, 46281.99725536]],

       [[50871.99698391, 53421.99683311, 56226.99666722],
        [50361.99701407, 52656.99687835, 55461.99671246],
        [49851.99704423, 52401.99689343, 55206.99672754],
        ...,
        [37866.99775301, 41691.99752681, 45006.99733076],
        [37866.99775301, 41691.99752681, 45006.99733076],
        [38886.99769269, 42711.99746649, 46026.99727044]],

       [[50106.99702915, 52911.99686327, 55971.9966823 ],
        [49596.99705931, 52146.99690851, 55206.99672754],
        [49341.99707439, 52146.99690851, 55206.99672754],
        ...,
        [37611.99776809, 41181.99755697, 44496.99736092],
        [37611.99776809, 4118

In [13]:
test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
# test_imageinteger_only.tflite

array([[[[-76, -84, -93],
         [-73, -81, -91],
         [-73, -81, -91],
         ...,
         [-24, -39, -52],
         [-23, -38, -51],
         [-28, -42, -55]],

        [[-73, -83, -94],
         [-71, -80, -91],
         [-69, -79, -90],
         ...,
         [-22, -37, -50],
         [-22, -37, -50],
         [-26, -41, -54]],

        [[-70, -81, -93],
         [-68, -78, -90],
         [-67, -78, -90],
         ...,
         [-21, -35, -48],
         [-21, -35, -48],
         [-24, -39, -52]],

        ...,

        [[ 57,  83,  96],
         [ 46,  70,  82],
         [ 24,  46,  58],
         ...,
         [ 37,  58,  74],
         [ 41,  61,  78],
         [ 43,  63,  80]],

        [[ 46,  67,  82],
         [ 39,  61,  75],
         [ 52,  75,  87],
         ...,
         [ 35,  56,  73],
         [ 35,  57,  73],
         [ 36,  57,  74]],

        [[ 46,  62,  79],
         [ 47,  66,  81],
         [ 64,  86,  99],
         ...,
         [ 27,  48,  66],
        

In [25]:
def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor


interpreter.set_tensor(input_details["index"], test_image)
interpreter.invoke()
boxes = get_output_tensor(interpreter, 1)
classes = get_output_tensor(interpreter, 3)
scores = get_output_tensor(interpreter, 0)
count = int(get_output_tensor(interpreter, 2))


output_scale, output_zero_point  = interpreter.get_output_details()[0]['quantization']
output = (scores[scores.argmax()] - output_zero_point) * output_scale
output

0.5208243161905557